In [3]:
import os
from tqdm import tqdm
import shutil

In [4]:
DATASET_DIR = "openimages"
ANNOTATIONS_FILE = os.path.join(DATASET_DIR, "train-annotations-object-segmentation.csv")
ALL_MASKS_DIR = os.path.join(DATASET_DIR, "all_masks")
MASKS_DIR = os.path.join(DATASET_DIR, "masks")
IMAGES_DIR = os.path.join(DATASET_DIR, "images")
READ_RATIO = .002
READ_EVERY = int(1/READ_RATIO)
IMAGE_LIST_FILE = os.path.join(DATASET_DIR, "download_list.txt")
OUTPUT_DIR = os.path.join(DATASET_DIR, "transparent_images")

In [5]:
FILE_RANGE = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "a", "b", "c", "d", "e", "f"]

In [6]:
os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(MASKS_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(ALL_MASKS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:
!wget https://storage.googleapis.com/openimages/v5/train-annotations-object-segmentation.csv -O {ANNOTATIONS_FILE}

--2024-10-20 13:11:44--  https://storage.googleapis.com/openimages/v5/train-annotations-object-segmentation.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.36.123, 142.251.36.91, 142.251.37.123, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.36.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 806943628 (770M) [text/csv]
Saving to: ‘openimages/train-annotations-object-segmentation.csv’

openimages/train-an 100%[===================>] 769.56M  31.9MB/s    in 32s     

2024-10-20 13:12:17 (23.7 MB/s) - ‘openimages/train-annotations-object-segmentation.csv’ saved [806943628/806943628]



In [7]:
for digit in FILE_RANGE:
    !wget https://storage.googleapis.com/openimages/v5/train-masks/train-masks-{digit}.zip -O {DATASET_DIR}/train_masks_{digit}.zip

--2024-10-20 12:45:26--  https://storage.googleapis.com/openimages/v5/train-masks/train-masks-0.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.37.123, 142.251.36.91, 142.251.36.123, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.37.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 385547550 (368M) [application/zip]
Saving to: ‘openimages/train_masks_0.zip’

openimages/train_ma 100%[===================>] 367.69M  25.6MB/s    in 15s     

2024-10-20 12:45:42 (25.2 MB/s) - ‘openimages/train_masks_0.zip’ saved [385547550/385547550]

--2024-10-20 12:45:42--  https://storage.googleapis.com/openimages/v5/train-masks/train-masks-1.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.36.91, 142.251.37.123, 142.251.36.155, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.36.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 298431035 (285M) [appl

In [ ]:
for digit in tqdm(FILE_RANGE, total=len(FILE_RANGE)):
    !unzip -n openimages/train_masks_{digit}.zip -d {ALL_MASKS_DIR}

  0%|          | 0/16 [00:00<?, ?it/s]

Archive:  openimages/train_masks_0.zip


  6%|▋         | 1/16 [52:49<13:12:24, 3169.64s/it]

Archive:  openimages/train_masks_1.zip


 12%|█▎        | 2/16 [52:53<5:05:00, 1307.21s/it] 

...

Archive:  openimages/train_masks_3.zip


 25%|██▌       | 4/16 [1:45:34<5:35:29, 1677.49s/it]

Archive:  openimages/train_masks_4.zip


 31%|███▏      | 5/16 [4:52:40<15:38:49, 5120.87s/it]

Archive:  openimages/train_masks_5.zip


In [9]:
print(f"Reading every {READ_EVERY} entries")
image_mask_dict = dict()
with open(ANNOTATIONS_FILE) as f:
    for i, line in tqdm(enumerate(f.readlines())):
        if i == 0 or i % READ_EVERY != 0:
            continue
        data = line.split(",")
        mask_path = data[0]
        image_id = data[1]
        image_mask_dict[image_id] = mask_path
print(f"Read {len(image_mask_dict.keys())} entries")

Reading every 500 entries


2686667it [00:01, 2120766.31it/s]

Read 5349 entries


In [10]:
with open(IMAGE_LIST_FILE, "w+") as f:
    f.write("\n".join([f"train/{key}" for key in image_mask_dict.keys()]))

In [7]:
images = []
with open(IMAGE_LIST_FILE) as f:
    for line in f.readlines():
        images.append(line.replace("train/", ""))

In [8]:
image_mask_dict = dict()
with open(ANNOTATIONS_FILE) as f:
    for i, line in tqdm(enumerate(f.readlines())):
        data = line.split(",")
        mask_path = data[0]
        image_id = data[1]
        if image_id in images:
            image_mask_dict[image_id] = mask_path

443938it [00:16, 27676.41it/s]


KeyboardInterrupt: 

In [11]:
!wget https://raw.githubusercontent.com/openimages/dataset/master/downloader.py -O {DATASET_DIR}/downloader.py

--2024-10-20 13:13:37--  https://raw.githubusercontent.com/openimages/dataset/master/downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4244 (4.1K) [text/plain]
Saving to: ‘openimages/downloader.py’

openimages/download 100%[===================>]   4.14K  --.-KB/s    in 0s      

2024-10-20 13:13:38 (43.7 MB/s) - ‘openimages/downloader.py’ saved [4244/4244]



In [ ]:
!pip install boto3

In [ ]:
!python {DATASET_DIR}/downloader.py {IMAGE_LIST_FILE} --download_folder={IMAGES_DIR} --num_processes=5

In [11]:
shutil.rmtree(MASKS_DIR)
for mask_file in tqdm(image_mask_dict.values()):
    shutil.copy(os.path.join(ALL_MASKS_DIR, mask_file), os.path.join(MASKS_DIR, mask_file))

100%|██████████| 5349/5349 [04:09<00:00, 21.40it/s]


In [42]:
import cv2
import numpy as np
def create_transparent_png(image_path, mask_path, output_path):
    # Load the image and mask
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)  # Load image with channels (RGBA if needed)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)    # Binary mask (grayscale)
    
    if image is None:
        print(f"Image {image_path} not found")
        return False
    if mask is None:
        print(f"Mask {mask_path} not found")
        return False
    if len(image.shape) < 3:
        print(f"Image {image_path} weird")
        return False
    
    mask = cv2.resize(mask, (image.shape[1], image.shape[0]))
    
    # Ensure the image has 3 channels (RGB)
    if image.shape[2] == 3:
        # Add an alpha channel (transparency layer)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
        
    
    # Find the extreme points of the mask
    coords = cv2.findNonZero(mask)
    x, y, w, h = cv2.boundingRect(coords)

    # Crop the image and mask to the bounding box
    image = image[y:y+h, x:x+w]
    mask = mask[y:y+h, x:x+w]

    # Create a transparent background using the mask
    # Set fully transparent where mask is black (0) and fully opaque where mask is white (255)
    image[:, :, 3] = mask

    # Save the image with a transparent background
    cv2.imwrite(output_path, image)
    return True

In [43]:
distractors = []
for i, (image, mask) in tqdm(enumerate(image_mask_dict.items()), total=len(image_mask_dict)):
    output_path = os.path.join(OUTPUT_DIR, f"transparent_image{i}.png")
    if create_transparent_png(os.path.join(IMAGES_DIR, f"{image}.jpg"), 
                           os.path.join(MASKS_DIR, f"{mask}"), 
                           output_path):
        distractors.append(output_path)

  2%|▏         | 83/5349 [00:01<01:36, 54.31it/s]

Image openimages/images/0ad5bc2c5a769803.jpg weird


  2%|▏         | 126/5349 [00:02<01:31, 57.13it/s]

Image openimages/images/9159b8d542807d03.jpg weird


  3%|▎         | 156/5349 [00:02<01:18, 66.05it/s]

...

Image openimages/images/209b5018a25bc38c.jpg not found
Image openimages/images/82d7c7a7ff756d96.jpg not found
Image openimages/images/dacf41f5adb213a5.jpg not found
Image openimages/images/6b5a6dc582892722.jpg not found
Image openimages/images/c5a01e5aa60a278d.jpg not found
Image openimages/images/bdc0f69b75e6c6c9.jpg not found
Image openimages/images/f9b14ba7b20551d8.jpg not found
Image openimages/images/ead0c6415a599b61.jpg not found
Image openimages/images/dc25834e4879c90d.jpg not found
Image openimages/images/a293b53b6823e9d2.jpg not found
Image openimages/images/d6a400283401f39c.jpg not found
Image openimages/images/a8f126c5df7df9ab.jpg not found
Image openimages/images/f72fd3fa0a24a138.jpg not found
Image openimages/images/37a12aea074fe80c.jpg not found
Image openimages/images/e2f3cd58dd23adbc.jpg not found
Image openimages/images/c4e8965125a7b340.jpg not found
Image openimages/images/38fb064897c1a473.jpg not found
Image openimages/images/a42627959504ebc4.jpg not found
Image open

[ WARN:0@1808.225] global loadsave.cpp:241 findDecoder imread_('openimages/images/5043fe88f5b85e04.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.228] global loadsave.cpp:241 findDecoder imread_('openimages/images/8dd94d4857787770.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.231] global loadsave.cpp:241 findDecoder imread_('openimages/images/aa0c32be804f64db.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.233] global loadsave.cpp:241 findDecoder imread_('openimages/images/b08b9e662ab2e456.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.236] global loadsave.cpp:241 findDecoder imread_('openimages/images/541479955e339303.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.238] global loadsave.cpp:241 findDecoder imread_('openimages/images/8efd6d3e44adc654.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.242] global loadsave.cpp:241 findDecoder imread_('openimages/i

Image openimages/images/8efd6d3e44adc654.jpg not found
Image openimages/images/a9f7377eb2c85869.jpg not found
Image openimages/images/f316e12331fea2f3.jpg not found
Image openimages/images/34bc2d81d7544c4e.jpg not found
Image openimages/images/4b711102cd53cb11.jpg not found
Image openimages/images/0a5c24f87e9df87e.jpg not found
Image openimages/images/e3566d338b027b89.jpg not found
Image openimages/images/1f29f089ad8f14ac.jpg not found
Image openimages/images/cb5697334723cf16.jpg not found
Image openimages/images/a9b517ee497b74c6.jpg not found
Image openimages/images/90e34bf5d0b1fa32.jpg not found
Image openimages/images/0b6c21d2dbc11a66.jpg not found
Image openimages/images/8e076f9d8996ee0b.jpg not found
Image openimages/images/34262b3028861363.jpg not found
Image openimages/images/5687cfa7276afca8.jpg not found
Image openimages/images/a2e94326aafa9383.jpg not found
Image openimages/images/ee6f03f3f1d9ecc8.jpg not found
Image openimages/images/ad2917232f9988cf.jpg not found
Image open

[ WARN:0@1808.429] global loadsave.cpp:241 findDecoder imread_('openimages/images/4646d4e70d73b1b8.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.432] global loadsave.cpp:241 findDecoder imread_('openimages/images/07851cb3271ee5f2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.435] global loadsave.cpp:241 findDecoder imread_('openimages/images/e5386621914494d1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.438] global loadsave.cpp:241 findDecoder imread_('openimages/images/e7b8475049c5d21f.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.440] global loadsave.cpp:241 findDecoder imread_('openimages/images/73c708515baeae1f.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.441] global loadsave.cpp:241 findDecoder imread_('openimages/images/fa5ac0aa8a6bcab6.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1808.444] global loadsave.cpp:241 findDecoder imread_('openimages/i

Image openimages/images/a404adef226e5536.jpg not found
Image openimages/images/1801d4cb137208e5.jpg not found
Image openimages/images/5dc27cc7f16f6ce0.jpg not found
Image openimages/images/76a5b88169ed2f47.jpg not found
Image openimages/images/cc00605484bb2d4c.jpg not found
Image openimages/images/c89ef4a7efed8336.jpg not found
Image openimages/images/a9ae9a98b6d489f9.jpg not found
Image openimages/images/67f0e9fb67c4a7c7.jpg not found
Image openimages/images/ed45056cc393342d.jpg not found
Image openimages/images/11101039fb4899b7.jpg not found
Image openimages/images/b35934bd5f0b6eca.jpg not found
Image openimages/images/041654a5c555f9a8.jpg not found
Image openimages/images/20ff4dd67979e7e3.jpg not found
Image openimages/images/f370e4a47dfdfa4a.jpg not found
Image openimages/images/489a446548925b55.jpg not found
Image openimages/images/ee9976b5d045db98.jpg not found
Image openimages/images/809a031571f0ab2a.jpg not found
Image openimages/images/f75a8fa5b69942d5.jpg not found
Image open

In [7]:
!ls {OUTPUT_DIR} | wc -l

866


In [14]:
!mkdir -p "synthetic-dataset-generation/data/distractors/openimage"
!cp {OUTPUT_DIR}/* synthetic-dataset-generation/data/distractors/openimage

In [17]:
!mkdir synthetic-dataset-generation/data/objects/leaf
!cp us_leaf_alpha/* synthetic-dataset-generation/data/objects/leaf

In [27]:
!cp street_view/* synthetic-dataset-generation/data/backgrounds

In [7]:
import json
with open("synthetic-dataset-generation/data/objects/splits.json", "w+") as f:
    elements =  [os.path.join("leaf", file) for file in os.listdir("synthetic-dataset-generation/data/objects/leaf")]
    content = json.dumps({
        "test": elements,
        "train": elements,
        "validation": elements
    })
    f.write(content)

In [8]:
import json
with open("synthetic-dataset-generation/data/distractors/splits.json", "w+") as f:
    elements = [os.path.join("openimage", file) for file in os.listdir("synthetic-dataset-generation/data/distractors/openimage")]
    content = json.dumps({
        "test": elements,
        "train": elements,
        "validation": elements
    })
    f.write(content)

In [9]:
import json
with open("synthetic-dataset-generation/data/backgrounds/splits.json", "w+") as f:
    elements = [file for file in os.listdir("synthetic-dataset-generation/data/backgrounds") if file.endswith('.jpg')]
    content = json.dumps({
        "test": elements,
        "train": elements,
        "validation": elements
    })
    f.write(content)

In [25]:
!cd synthetic-dataset-generation && pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/lamhoangtung/pyblur.git to /tmp/pip-req-build-w3fe09os
  Running command git clone -q https://github.com/lamhoangtung/pyblur.git /tmp/pip-req-build-w3fe09os
  Resolved https://github.com/lamhoangtung/pyblur.git to commit d2d3663fc2cf7eb939ffd6ffc6471ef74de32695
     |████████████████████████████████| 34.6 MB 500 kB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 25.0 MB/s eta 0:00:01
     |████████████████████████████████| 26.9 MB 65.2 MB/s eta 0:00:01
     |████████████████████████████████| 55.0 MB 165 kB/s eta 0:00:01
     |████████████████████████████████| 3.7 MB 60.8 MB/s eta 0:00:01
     |████████████████████████████████| 240 kB 63.1 MB/s eta 0:00:01
     |████████████████████████████████| 316 kB 62.4 MB/s eta 0:00:01
     |████████████████████████████████| 124 kB 54.8 MB/s eta 0:00:01
     |████████████████████████████████| 43.9 MB 55.1 MB/s eta 0:00:01
     |██

In [ ]:
!cd synthetic-dataset-generation && python src/tools/generate_synthetic_data.py